
*Note:* You can run this from your computer (Jupyter or terminal), or use one of the
hosted options:

[![binder-logo](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ploomber/binder-env/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fploomber%252Fprojects%26urlpath%3Dlab%252Ftree%252Fprojects%252Fserialization%252FREADME.ipynb%26branch%3Dmaster)

[![deepnote-logo](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?template=deepnote&url=https://github.com/ploomber/projects/blob/master/serialization/README.ipynb)


# Serialization

Incremental builds allow Ploomber to skip tasks whose source code hasn't changed; each task must save their products to disk to enable such a feature. However, there are some cases when we don't want our pipeline to perform disk operations. For example, if we're going to deploy our pipeline, eliminating disk operations reduces runtime considerably.

To enable a pipeline to work in both disk-based and in-memory scenarios, we can declare a `serializer` and `unzerializer` in our pipeline declaration, effectively separating our task's logic from the read/write logic.

Note that this only applies to function tasks; other tasks are unaffected by the `serializer`/`unserializer` configuration.

In [1]:
from ploomberutils import display_file

## Built-in pickle serialization

The easiest way to get started is to use the built-in serializer and unserializer, which use the `pickle` module.

Let's see an example; the following pipeline has two tasks. The first one generates a dictionary, and the second one creates two dictionaries. Since we are using the pickle-based serialization, each dictionary is saved in the pickle binary format:

In [2]:
display_file('simple.yaml')


```yaml
serializer: ploomber.io.serializer_pickle
unserializer: ploomber.io.unserializer_pickle

tasks:
  - source: tasks.first
    product: output/one_dict
  
  - source: tasks.second
    product:
        another: output/another_dict
        final: output/final_dict
```


Let's take a look at the task's source code:

In [3]:
display_file('tasks.py')


```py
def first():
    return dict(a=1, b=2)


def second(upstream):
    first = upstream['first']
    another = dict(a=first['b'] + 1, b=first['a'] + 1)
    final = dict(a=100, b=200)
    return dict(another=another, final=final)
```


Since we configured a `serializer` and `unserializer`, function tasks must `return` their outpues instead of saving them to disk in the function's body.

`first` does not have any upstream dependencies and returns a dictionary. `second` has the previous task as dependency and returns two dictionaries. Note that the keys in the returned dictionary must match the names of the products declared in `pipeline.yaml` (`another`, `final`).

Let's now run the pipeline.

In [4]:
%%sh
ploomber build --entry-point simple.yaml --force

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
first   True         0.000606       28.1337
second  True         0.001548       71.8663


Building task 'second': 100%|██████████| 2/2 [00:06<00:00,  3.40s/it]


The pickle format has important [security concerns](https://docs.python.org/3/library/pickle.html), **remember only to unpickle data you trust**.

## Custom serialization logic

We can also define our own serialization logic, by using the `@serializer`, and `@unserializer` decorators. Let's replicate what our pickle-based serializer/unserializer is doing as an example:

In [5]:
display_file('custom.py', symbols=['my_pickle_serializer', 'my_pickle_unserializer'])


```py
from pathlib import Path
import pickle

from ploomber.io import serializer, unserializer


@serializer()
def my_pickle_serializer(obj, product):
    Path(product).write_bytes(pickle.dumps(obj))


@unserializer()
def my_pickle_unserializer(product):
    return pickle.loads(Path(product).read_bytes())
```


A `@serializer` function must take two arguments: the object to serializer and the product object (taken from the task declaration). The `@unserializer` must take a single argument (the product to unserializer), and return the unserializer object.

Let's modify our original pipeline to use this serializer/unserializer:

In [6]:
display_file('custom.yaml')


```yaml
serializer: custom.my_pickle_serializer
unserializer: custom.my_pickle_unserializer

tasks:
  - source: tasks.first
    product: output/one_dict
  
  - source: tasks.second
    product:
        another: output/another_dict
        final: output/final_dict
```


In [7]:
%%sh
ploomber build --entry-point custom.yaml --force

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
first   True         0.000577        27.969
second  True         0.001486        72.031


Building task 'second': 100%|██████████| 2/2 [00:06<00:00,  3.21s/it]


## Custom serialization logic based on the product's extension

Under many circumstances, there are more suitable formats than pickle. For example, we may want to store lists or dictionaries as JSON files and other files using pickle. The `@serializer`/`@unserializer` decorators use mapping as the first argument to dispatch to different functions depending on the product's extension. Let's see an example:

In [8]:
display_file('custom.py', symbols=['write_json', 'read_json', 'my_serializer', 'my_unserializer'])


```py
from pathlib import Path
import pickle
import json

from ploomber.io import serializer, unserializer


def write_json(obj, product):
    Path(product).write_text(json.dumps(obj))


def read_json(product):
    return json.loads(Path(product).read_text())


@serializer({'.json': write_json})
def my_serializer(obj, product):
    Path(product).write_bytes(pickle.dumps(obj))


@unserializer({'.json': read_json})
def my_unserializer(product):
    return pickle.loads(Path(product).read_bytes())
```


Let's modify our example pipeline. The product in the first task does not have an extension (`output/one_dict`), hence, it will use pickle-based logic. However, the tasks in the second task have a `.json` extension, and will be saved as JSON files.

In [9]:
display_file('with-json.yaml')


```yaml
serializer: custom.my_serializer
unserializer: custom.my_unserializer

tasks:
  - source: tasks.first
    product: output/one_dict
  
  - source: tasks.second
    product:
        another: output/another_dict.json
        final: output/final_dict.json
```


In [10]:
%%sh
ploomber build --entry-point with-json.yaml --force

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
first   True         0.000614       27.1802
second  True         0.001645       72.8198


Building task 'second': 100%|██████████| 2/2 [00:06<00:00,  3.22s/it]


Let's print the `.json` files to verify they're not pickle files:

In [11]:
display_file('output/another_dict.json')


```json
{"a": 3, "b": 2}
```


In [12]:
display_file('output/final_dict.json')


```json
{"a": 100, "b": 200}
```


## Using a fallback format

Since it's common to have a `fallback` serialization format, the decorators have a `fallback` argument that, when enabled, uses the `pickle` module when the product's extension does not match any of the registered ones in the first argument.

The example works the same as the previous one, except we don't have to write our pickle-based logic.

`fallback` can also take the [joblib](https://github.com/joblib/joblib) or [cloudpickle](https://github.com/cloudpipe/cloudpickle) values. They're similar to the pickle format but have some advantages. For example, `joblib` produces smaller files when the serialized object contains many NumPy arrays, while cloudpickle supports serialization of some objects that the pickle module doesn't. To use `fallback='joblib'` or `fallback='cloudpickle'` the corresponding module must be installed.

In [13]:
display_file('custom.py', symbols=['my_fallback_serializer', 'my_fallback_unserializer'])


```py
from ploomber.io import serializer, unserializer


@serializer({'.json': write_json}, fallback=True)
def my_fallback_serializer(obj, product):
    pass


@unserializer({'.json': read_json}, fallback=True)
def my_fallback_unserializer(product):
    pass
```


In [14]:
display_file('fallback.yaml')


```yaml
serializer: custom.my_fallback_serializer
unserializer: custom.my_fallback_unserializer

tasks:
  - source: tasks.first
    product: output/one_dict
  
  - source: tasks.second
    product:
        another: output/another_dict.json
        final: output/final_dict.json
```


In [15]:
%%sh
ploomber build --entry-point fallback.yaml --force

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
first   True         0.000572       24.2168
second  True         0.00179        75.7832


Building task 'second': 100%|██████████| 2/2 [00:06<00:00,  3.34s/it]


Let's print the JSON files to verify their contents:

In [16]:
display_file('output/another_dict.json')


```json
{"a": 3, "b": 2}
```


In [17]:
display_file('output/final_dict.json')


```json
{"a": 100, "b": 200}
```


## Using default serializers

Ploomber comes with a few convenient serialization functions to write more succint serializers. We can request the use of such default serializers using the `defaults` argument, which takes a list of extensions:

In [18]:
display_file('custom.py', symbols=['my_defaults_serializer', 'my_defaults_unserializer'])


```py
from ploomber.io import serializer, unserializer


@serializer(fallback=True, defaults=['.json'])
def my_defaults_serializer(obj, product):
    pass


@unserializer(fallback=True, defaults=['.json'])
def my_defaults_unserializer(product):
    pass
```


Here we're asking to dispatch `.json` products and use `pickle` for all other extensions, the same as we did for the previous examples, except this time, we don't have to pass the mapping argument to the decorators.

`defaults` support:

1. `.json`: the returned object must be JSON-serializable (e.g., a list or a dictionary)
2. `.txt`: the returned object must be a string
3. `.csv`: the returned object must be a `pandas.DataFrame`
4. `.parquet`: the returned object must be a `pandas.DataFrame,` and a parquet library should be installed (such as `pyarrow`).

In [19]:
display_file('defaults.yaml')


```yaml
serializer: custom.my_defaults_serializer
unserializer: custom.my_defaults_unserializer

tasks:
  - source: tasks.first
    product: output/one_dict
  
  - source: tasks.second
    product:
        another: output/another_dict.json
        final: output/final_dict.json
```


In [20]:
%%sh
ploomber build --entry-point defaults.yaml --force

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
first   True         0.000553       25.3553
second  True         0.001628       74.6447


Building task 'second': 100%|██████████| 2/2 [00:06<00:00,  3.22s/it]


Let's print the JSON files to verify their contents:

In [21]:
display_file('output/another_dict.json')


```json
{"a": 3, "b": 2}
```


In [22]:
display_file('output/final_dict.json')


```json
{"a": 100, "b": 200}
```


## Wrapping up

Configuring a `serializer` and `unserializer` in your `pipeline.yaml` is optional, but it helps you quickly generate a fully in-memory pipeline for serving predictions.

If you want to learn more about in-memory pipelines, check out the [following guide.](https://ploomber.readthedocs.io/en/latest/user-guide/deployment.html#online-service-api).

For a complete example showing how to manage a training and a serving pipeline, and deploy it as a Flask API, [click here.](https://github.com/ploomber/projects/tree/master/ml-online).